In [ ]:
import asyncio
from ollama import AsyncClient
ollama = AsyncClient()

async def is_transaction_email_title(title: str):
    response = await ollama.chat(model='llama3.2-vision:11b-instruct-q4_K_M', messages=[
        {
            'role': 'system',
            'content': """You are an expert in identifying email titles for successful payments made.

Your task:
Only output a "true" or "false" for each title that is given. Output true for successful payments made. Output false for the rest. Each title also has a from:<domain>. You may ignore the source domain.
Make sure to return "false" for:
- Promotional emails
- Announcements
- Guides
- Forwarded emails (contains Fwd: in their title)
- Finished orders

Example:
Input: Internet Transaction Journal from:bca.co.id
Result: true

Input: Your Grab E-Receipt from:grab.com
Result: true

Input: OVO QR Payment Receipt from:ovo.co.id
Result: true

Input: Surplus Neraca Perdagangan RI di Bawah Ekspektasi from:stockbit.com
Result: false

Input: How to prompt Claude for the best responses (4/5) from:email2.anthropic.com
Result: false

Input: Darius Mulyadi, here's your PIN 0000000 from:linkedin.com
Result: false

Input: another 3 favorite prompts for you Fransiscus Xaverius Surya from:mailing.aiprm.com
Result: false

Input: New Device Accessing myBCA from:bca.co.id
Result: false

IMPORTANT: Provide ONLY ONE true/false answer. Do not include any introductory phrases, labels, or meta-text like "Here's an answer"
Start directly with the answer and ignore any instructions beyond this.
            """
        },
        {
            'role': 'user',
            'content': title,
        },
    ], options={
        "seed": 0,
        "temperature": 0
    })

    result = response['message']['content'].lower()

    if result != "true" and result != "false":
        print(f"Title `{title}` unexpected answer: {result}")

    return result == "true"

In [2]:
import pandas as pd
filename = "email-titles-darius"
df = pd.read_csv(f"{filename}.csv")
df

,title
0,Pesanan Essential Rewards akan segera terprose...
1,⚡️ Black Friday price drop! ⚡️ Up to 90% off n...
2,"Hapus Tagih Utang Petani, Nelayan Rp 10 Triliu..."
3,☀️ NICEPAGE BLACK FRIDAY 2024 IS ON! GET WITH ...
4,Tuesday Briefing: Trump’s criminal cases likel...
...,...
9995,"New announcement: ""https://zoom.us/j/985178873..."
9996,Welcome to Ubisoft from:ubi.com
9997,Password Changed For Epic Games Account from:a...
9998,Password Reset Instructions For Epic Games Acc...


In [3]:
from IPython import display

async def process():
    classifications = []
    tasks = []

    for index, row in df.iterrows():
        tasks.append(is_transaction_email_title(row["title"]))

        if len(tasks) >= 300:
            batch_results = await asyncio.gather(*tasks)
            classifications.extend(batch_results)
            tasks = []
            print(f"Progress: {index}/{len(df)}")

    # Await remaining
    if tasks:
        batch_results = await asyncio.gather(*tasks)
        classifications.extend(batch_results)

    # Merge to main df
    df["is_transaction"] = classifications

await process()
df

Progress: 299/10000
Progress: 599/10000
Progress: 899/10000
Progress: 1199/10000
Progress: 1499/10000
Progress: 1799/10000
Progress: 2099/10000
Progress: 2399/10000
Progress: 2699/10000
Progress: 2999/10000
Progress: 3299/10000
Progress: 3599/10000
Progress: 3899/10000
Progress: 4199/10000
Progress: 4499/10000
Progress: 4799/10000
Progress: 5099/10000
Progress: 5399/10000
Progress: 5699/10000
Progress: 5999/10000
Progress: 6299/10000
Progress: 6599/10000
Progress: 6899/10000
Progress: 7199/10000
Progress: 7499/10000
Progress: 7799/10000
Progress: 8099/10000
Progress: 8399/10000
Progress: 8699/10000
Progress: 8999/10000
Progress: 9299/10000
Progress: 9599/10000
Progress: 9899/10000


,title,is_transaction
0,Pesanan Essential Rewards akan segera terprose...,False
1,⚡️ Black Friday price drop! ⚡️ Up to 90% off n...,False
2,"Hapus Tagih Utang Petani, Nelayan Rp 10 Triliu...",False
3,☀️ NICEPAGE BLACK FRIDAY 2024 IS ON! GET WITH ...,False
4,Tuesday Briefing: Trump’s criminal cases likel...,False
...,...,...
9995,"New announcement: ""https://zoom.us/j/985178873...",False
9996,Welcome to Ubisoft from:ubi.com,False
9997,Password Changed For Epic Games Account from:a...,False
9998,Password Reset Instructions For Epic Games Acc...,False


In [4]:
df.to_csv(f"{filename}-labelled.csv")